In [1]:
# coding: utf-8
import numpy as np
import tensorflow as tf

In [2]:
C = 10 # classes num
N = 784 # features num
learning_rate = 0.01
epochs = 2
batch_size = 128
num_steps = 2000
display_step = 100

In [3]:
def model_net(X, n_classes, n_features):
    W = tf.Variable(tf.zeros([n_features, n_classes]), name='W')
    b = tf.Variable(tf.zeros(n_classes), name='b')

    logits = tf.matmul(X, W) + b
    return logits

In [15]:
def loss_fn(logits, labels, l2_reg=0):
    logits_loss = tf.losses.sparse_softmax_cross_entropy(labels, logits=logits)
    W = tf.get_default_graph().get_tensor_by_name('W:0')
    l2_loss = l2_reg * tf.nn.l2_loss(W)
    return logits_loss + l2_loss, logits_loss, l2_loss

In [5]:
def accuracy_fn(logits, labels):
    predictions = tf.nn.softmax(logits)
    is_correct = tf.equal(tf.argmax(predictions, axis=1), labels)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    return accuracy

In [6]:
def train_input_fn(features, labels, batch_size, shuffle_size=1000):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(shuffle_size).repeat().batch(batch_size).prefetch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
x_train, y_train = mnist.train.images, mnist.train.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
class CumulateAvg:
    def __init__(self, var_list, display_step=100):
        self.var_list = var_list
        self.reset()
        self.display_step = display_step
        self.step = 0
        
    def reset(self):
        self.var_dict = {k: 0 for k in self.var_list}
        
    def avg_var(self):
        if self.step > 0:
            self.var_dict = {k: v / self.display_step for k, v in self.var_dict.items()}
        
    def cumulate(self, var_values):
        for i, v in enumerate(var_values):
            self.var_dict[self.var_list[i]] += v
        if self.step == 0 or (self.step + 1) % self.display_step == 0:
            self.avg_var()
            print_str = 'step: %d' % (self.step + 1)
            print_str += ''.join([', %s: %.4f' % (k, self.var_dict[k]) for k in self.var_list])
            print(print_str)
        self.step += 1

In [18]:
with tf.Graph().as_default() as g:
    with tf.Session().as_default() as sess:
        dataset = train_input_fn(x_train, y_train, batch_size)
        X, y = dataset
        X = tf.cast(X, tf.float32)
        y = tf.cast(y, tf.int64)
        
        logits = model_net(X, C, N)
        loss, logits_loss, l2_loss = loss_fn(logits, y, l2_reg=0.05)
        accuracy = accuracy_fn(logits, y)
        train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        sess.run(tf.global_variables_initializer())

        var_list = ['loss', 'accuracy', 'logits_loss', 'l2_loss']
        cumavg_obj = CumulateAvg(var_list, 100)
        for step in range(num_steps):
            res_list = sess.run([train_op, *[eval(x) for x in var_list]])
            cumavg_obj.cumulate(res_list[1:])

step: 1, loss: 2.3026, accuracy: 0.1172, logits_loss: 2.3026, l2_loss: 0.0000
step: 100, loss: 1.8991, accuracy: 0.6779, logits_loss: 1.8925, l2_loss: 0.0066
step: 200, loss: 1.4313, accuracy: 0.8008, logits_loss: 1.3987, l2_loss: 0.0326
step: 300, loss: 1.2263, accuracy: 0.8207, logits_loss: 1.1629, l2_loss: 0.0634
step: 400, loss: 1.0907, accuracy: 0.8346, logits_loss: 0.9987, l2_loss: 0.0920
step: 500, loss: 1.0060, accuracy: 0.8488, logits_loss: 0.8885, l2_loss: 0.1175
step: 600, loss: 0.9926, accuracy: 0.8460, logits_loss: 0.8542, l2_loss: 0.1384
step: 700, loss: 0.9834, accuracy: 0.8514, logits_loss: 0.8269, l2_loss: 0.1565
step: 800, loss: 0.9625, accuracy: 0.8493, logits_loss: 0.7905, l2_loss: 0.1720
step: 900, loss: 0.8970, accuracy: 0.8764, logits_loss: 0.7116, l2_loss: 0.1854
step: 1000, loss: 0.9382, accuracy: 0.8552, logits_loss: 0.7416, l2_loss: 0.1966
step: 1100, loss: 0.9346, accuracy: 0.8642, logits_loss: 0.7284, l2_loss: 0.2062
step: 1200, loss: 0.9281, accuracy: 0.86